In [26]:
import numpy as np
import pandas as pd

import lightgbm
import matplotlib.pylab as plt
import os

from sklearn.model_selection import train_test_split

Inserting all features from test into one dataframe

In [27]:
folder_train = 'features_train_csv'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)
print(features_train)
        
        

       average_income_per_grunnkrets  distance_store_busstop  area_grunnkrets
0                           561700.0              585.354489         0.155779
1                           555720.0              138.798366         0.264278
2                           495900.0               41.099519         0.160152
3                           521240.0               56.693017         0.095029
4                           585360.0              229.077959         0.251070
...                              ...                     ...              ...
12854                       505640.0              150.004125         0.291337
12855                       396640.0              322.573926         0.137188
12856                       549120.0              174.851316         0.123431
12857                       498660.0              121.372663         0.034857
12858                       579780.0              189.983143         0.204915

[12859 rows x 3 columns]


Repeat for test features

In [28]:
folder_test = 'features_test_csv'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)
print(features_test)
    

      average_income_per_grunnkrets  distance_store_busstop  area_grunnkrets
0                          551600.0               25.630061         0.057027
1                          531240.0              111.736189         0.165993
2                          523300.0              404.825119         0.236628
3                          591380.0              317.141300         0.983436
4                          594020.0              101.273354         0.449502
...                             ...                     ...              ...
8572                       560700.0              364.034347         1.708924
8573                       593340.0              228.923169         0.310575
8574                       629100.0              136.232163         1.627789
8575                       590100.0              129.203732         1.103012
8576                       529980.0              212.494198         1.677433

[8577 rows x 3 columns]


Target train data:

In [29]:
target_train = pd.read_csv("data/stores_train.csv")
target_train = target_train['revenue'].values

Light GBM model:

In [30]:
lgbm_model = lightgbm.LGBMRegressor(
    num_leaves=4,
    max_depth=5, 
    random_state=42, 
    silent=True, 
    metric='mse',
    n_jobs=4, 
    n_estimators=1000,
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.05
)

In [31]:
#print(features_train)
#print(type(features_train))
print(len(features_train))
print(target_train)
print(type(target_train))
target_train = target_train
print(len(target_train))

12859
[17.998 23.828 16.099 ... 38.225  3.642  2.328]
<class 'numpy.ndarray'>
12859


In [32]:
lgbm_model.fit(features_train, target_train)
lgbm_preditions = lgbm_model.predict(features_test)

c:\Python310\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


In [33]:
print(lgbm_preditions)

[ 5.66628177 10.50847669  4.97765824 ... 12.8724819  10.19661478
 11.01592438]


In [34]:
stores_test = pd.read_csv("data/stores_test.csv")
stores_test_id = stores_test['store_id'].values

In [36]:
predictions = pd.DataFrame()

predictions['id'] = stores_test_id
predictions['predicted'] = lgbm_preditions
predictions.to_csv("predictions/lightGBM2.csv", index=False)
print(predictions)


                              id  predicted
0     914206820-914239427-717245   5.666282
1     916789157-916823770-824309  10.508477
2       913341082-977479363-2948   4.977658
3      889682582-889697172-28720   8.645958
4     997991699-998006945-417222  12.373533
...                          ...        ...
8572  917323003-917383529-844309   8.801378
8573  917353379-917411824-845904   8.959255
8574  917072302-917089248-833647  12.872482
8575  916960557-916993161-829908  10.196615
8576   987280891-972040746-45320  11.015924

[8577 rows x 2 columns]
